In [ ]:
from pathlib import Path
import sys, os
sys.path.append(str(Path.cwd().parent))

from simulator import StockSimulator
from policy import KNNPolicy

# Define the coin names to be used in the simulation
coin_names = [ 'DOGEUSDT', 'LINKUSDT', 'TONUSDT', 'SOLUSDT', 'BNBUSDT', 
    'PEPEUSDT', 'BTCUSDT', 'ADAUSDT', 'XRPUSDT', 'USDCUSDT'
]

In [ ]:
import optuna
import warnings
warnings.filterwarnings("ignore")
from functools import partial

# for each coin complete research
for cn in coin_names:

    def trial(trial: optuna.trial.Trial):
        # define hyperparameters of KNN
        k_neighbours = trial.suggest_categorical("k_neighbours", range(1, 10))
        weighted = trial.suggest_categorical("KNN_weighted", [True, False])
        distance_type = trial.suggest_categorical('distance_type', ['euclidean', 'manhattan'])

        day_window = trial.suggest_categorical('day_window', range(5, 21))
        
        # create simulator object and policy
        simulator = StockSimulator()
        policy = KNNPolicy(
            day_window=day_window, 
            k_neighbours=k_neighbours, 
            weighted=weighted, 
            metric=distance_type
        )
        
        # obtain metrics
        metrics = simulator.run_simulation(
            model=policy,
            coin_name=cn,
            path_to_storage='../data/',
            warm_start=300
        )
        
        # main goal is spp
        return metrics['profitability']['spp'] # Return simple percentage profitability

    try:
        sampler = optuna.samplers.TPESampler(n_startup_trials=10, group=True, multivariate=True)
        study = optuna.create_study(
            sampler=sampler,
            load_if_exists=True,
            storage="sqlite:///../optuna/KNN_db.sqlite3",
            direction="maximize", 
            study_name=cn)
        study.optimize(trial, n_trials=75, n_jobs=15, show_progress_bar=True)
    except: pass

/Users/artemon/.local/share/virtualenvs/MFT-project-lheyyFxX/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'coin_names' is not defined